## Protocole 3 : Étude de la Synchronisation entre Triggers et Stimuli Visuels

Ce protocole vise à mesurer le délai et la gigue entre un trigger envoyé par une carte Arduino et l'apparition effective d'un stimulus visuel sur un écran.

### Prérequis

**Matériel :**
*   Tout le matériel des protocoles 1 et 2

**Logiciels :**
*   Tous les logiciels et bibliothèques des protocoles 1 et 2.

### Étapes à suivre

#### 1. Génération des Triggers et des Stimuli

1.  **Disposez de deux écrans** : un pour la fenêtre d’enregistrement (Waveforms) et l’autre pour l’affichage des stimuli visuels.

2.  **Préparez le code .ino et le code .py** associés. L'ensemble génère des triggers sur un pin (ici le pin 8) à chaque présentation de stimuli visuels, en prenant en argument la période désirée.

> **Code Arduino (.ino) pour la réception de commandes série.**
>
> ```arduino
> #include <TimerOne.h>
> 
> const int triggerPin = 8;
> volatile bool doit_envoyer = false;
> 
> void setup() {
>   Serial.begin(115200);
>   pinMode(triggerPin, OUTPUT);
>   digitalWrite(triggerPin, LOW);  
> 
>   // Lancement du Timer toutes les 500 ms
>   Timer1.initialize(500000);  // 500 000 µs = 500 ms
>   Timer1.attachInterrupt(routine_periode);
> }
> 
> void loop() {
>   if (doit_envoyer) {
>     doit_envoyer = false;
> 
>     // 1. Envoi de la commande série
>     Serial.println("SHOW");
>     Serial.flush();  // assure la transmission complète avant TTL
> 
>     // 2. Envoi du trigger TTL de 1 ms
>     digitalWrite(triggerPin, HIGH);  
>     delayMicroseconds(1000);
>     digitalWrite(triggerPin, LOW); 
>   }
> }
> 
> // Fonction appelée toutes les 500 ms par le Timer hardware
> void routine_periode() {
>   doit_envoyer = true;
> }
> ```


3.  **Téléversez le code `.ino`** sur votre carte Arduino en suivant les étapes 2 à 5 du Protocole 1.

4.  **Préparez votre code Python (.py)** qui affiche les stimuli et envoie les commandes à l'Arduino.

> **Code Python (.py) pour l'affichage des stimuli et l'envoi des triggers.**
> ```python
> from expyriment import design, control, stimuli, misc
> import serial
> 
> # === Paramètres ===
> SERIAL_PORT = '/dev/ttyACM0'  # À adapter si nécessaire
> BAUDRATE = 115200
> SQUARE_DURATION = 100  # Durée en ms du carré blanc
> 
> # === Initialisation Arduino & Expyriment ===
> arduino = serial.Serial(SERIAL_PORT, BAUDRATE, timeout=0.1)
> 
> exp = design.Experiment(name="Square Display on Serial Trigger")
> control.set_develop_mode(True)
> control.initialize(exp)
> 
> square = stimuli.Rectangle((400, 400))
> blank = stimuli.BlankScreen()
> square.preload()
> blank.preload()
> 
> control.start(skip_ready_screen=True)
> 
> clock = misc.Clock()
> print("✅ Prêt. Attente des commandes série 'SHOW'...")
> 
> try:
>     while True:
>         if arduino.in_waiting:
>             line = arduino.readline().decode(errors="ignore").strip()
>             if line == "SHOW":
>                 square.present(update=True)
>                 clock.wait(SQUARE_DURATION)
>                 blank.present(update=True)
>         exp.keyboard.process_control_keys()
> except KeyboardInterrupt:
>     print("Interrompu manuellement.")
> finally:
>     arduino.close()
>     control.end()
> ```


5.  **Lancez le code .py** sur le terminal de votre ordinateur.
>![Scope settings](../pictures/arduino_trigger_master_timer.png)

6.  Vous devriez maintenant avoir une fenêtre présentant les stimuli. **Placez cette fenêtre sur l'écran secondaire**. La carte Arduino devrait maintenant générer des triggers sur le pin concerné.

#### 2. Mesure des Signaux

1.  **Connectez les sondes de l'oscilloscope :**
    *   **Canal 1 (Trigger Arduino) :**
        *   Broche GND de l'oscilloscope sur le GND de l'Arduino.
        *   Broche de mesure sur le **pin 8** de l'Arduino.
    *   **Canal 2 (Photodiode) :**
        *   Broche GND de l'oscilloscope sur le GND de la photodiode.
        *   Broche de mesure sur la sortie (Vout) de la photodiode.

2.  **Placez la photodiode** au niveau de la fenêtre où sont présentés les stimuli.

3.  Lancez l'acquisition avec le bouton **`Run`**. Vous devriez maintenant observer deux signaux sur Waveforms :
    *   **Canal 1 :** Un signal de pics/créneaux (triggers Arduino).
    *   **Canal 2 :** Un autre signal qui varie avec l'intensité lumineuse (photodiode).
    Les deux signaux doivent être des créneaux de même période, mais de rapport cyclique différent.
>![Scope settings](../pictures/acquisition_arduino_photodiode.png)

4.  **Ajustez les échelles**.
>![Scope settings](../pictures/acquisition_characteristics_two_channels.png)
#### 3. Enregistrement des Signaux

1.  Suivez la même procédure que dans les protocoles 1 et 2, en vous assurant d'enregistrer les deux canaux. Vous obtiendrez un fichier `.csv` à plusieurs colonnes.
>![Scope settings](../pictures/record_button.png)
>![Scope settings](../pictures/choice_of_recording_characteristics_two_channels_wav.png)

#### 4. Analyse des Signaux

1.  **Préparez votre code Python** pour l'étude du fichier `.wav` à deux colonnes. Ce code doit détecter les pics sur les deux signaux et analyser les intervalles entre un front montant du trigger Arduino et le pic de la photodiode correspondant.

> **Code Python (.py) pour l'analyse de synchronisation (Arduino et photodiode).**
> ```python
> import numpy as np
> import matplotlib
> matplotlib.use('Agg')
> import matplotlib.pyplot as plt
> from scipy.io import wavfile
> from scipy.signal import find_peaks
> import os
> 
> # === Paramètres ===
> NOM_FICHIER_WAV = 'arduino_trigger_master_timer_500ms_both_2.wav'
> DISTANCE_MIN_MS = 100.0
> 
> def analyser_signaux_wav(nom_fichier):
>     try:
>         sampling_rate, data = wavfile.read(nom_fichier)
>         if data.ndim != 2 or data.shape[1] < 2:
>             raise ValueError("Le fichier WAV doit avoir au moins 2 canaux (Arduino, Photodiode).")
> 
>         ttl = data[:, 0]
>         photodiode = data[:, 1]
> 
>         print(f"--- Analyse du fichier : {nom_fichier} ---")
>         print(f"Taux d'échantillonnage : {sampling_rate} Hz")
> 
>         # Seuils dynamiques, photodiode basée sur l'amplitude absolue
>         seuil_ttl = 0.5 * np.max(ttl)
>         signal_photo_abs = np.abs(photodiode)
>         seuil_photo = 0.8 * np.max(signal_photo_abs)
>         print(f"Seuil TTL : {seuil_ttl:.2f}, Seuil Photodiode : {seuil_photo:.2f}")
> 
>         min_distance_samples = int((DISTANCE_MIN_MS / 1000) * sampling_rate)
> 
>         # Détection des pics
>         ttl_peaks, _ = find_peaks(ttl, height=seuil_ttl, distance=min_distance_samples)
>         photo_peaks, _ = find_peaks(signal_photo_abs, height=seuil_photo, distance=min_distance_samples)
> 
>         def analyse_intervalles(peaks, label):
>             intervalles = np.diff(peaks) / sampling_rate * 1000  # ms
>             moyenne = np.mean(intervalles)
>             print(f"\n--- {label} ---")
>             print(f"Nombre de triggers : {len(peaks)}")
>             print(f"Intervalle moyen : {moyenne:.2f} ms")
>             print(f"Min : {np.min(intervalles):.2f}, Max : {np.max(intervalles):.2f}, Écart-type : {np.std(intervalles):.2f}")
> 
>             plt.figure(figsize=(10, 5))
>             plt.hist(intervalles, bins=40, edgecolor='black', alpha=0.8)
>             plt.axvline(moyenne, color='red', linestyle='--', label=f'Moyenne = {moyenne:.2f} ms')
>             plt.title(f'Intervalles entre triggers - {label}')
>             plt.xlabel("Durée (ms)")
>             plt.ylabel("Occurrences")
>             plt.legend()
>             os.makedirs('figures', exist_ok=True)
>             plt.savefig(f'figures/hist_intervalles_{label.lower().replace(" ", "_")}.png')
>             plt.close()
> 
>         analyse_intervalles(ttl_peaks, "Arduino")
>         analyse_intervalles(photo_peaks, "Photodiode")
> 
>         # Délai Arduino → Photodiode
>         matched_delays = []
>         for t in ttl_peaks:
>             deltas = photo_peaks - t
>             pos_deltas = deltas[deltas > 0]
>             if len(pos_deltas) > 0:
>                 delay = pos_deltas[0] / sampling_rate * 1000
>                 matched_delays.append(delay)
> 
>         matched_delays = np.array(matched_delays)
>         print("\n--- Délai entre trigger Arduino et détection visuelle ---")
>         print(f"Nombre d'appariements : {len(matched_delays)}")
>         print(f"Délai moyen : {np.mean(matched_delays):.2f} ms, Min : {np.min(matched_delays):.2f}, Max : {np.max(matched_delays):.2f}, Jitter : {np.std(matched_delays):.2f} ms")
> 
>         plt.figure(figsize=(10, 5))
>         plt.hist(matched_delays, bins=30, edgecolor='black', alpha=0.8)
>         plt.axvline(np.mean(matched_delays), color='red', linestyle='--', label=f'Moyenne = {np.mean(matched_del
> ```

2.  **Lancez le code**.
>![Scope settings](../pictures/analyse_and_draw_triggers_photodiode.png)
3.  Vous avez désormais affiché sur le terminal les caractéristiques des deux signaux et celles des intervalles de latence, ainsi que les histogrammes correspondants que vous pouvez retrouver au chemin d'accès renseigné.
>![Scope settings](../pictures/command_results_arduino_photodiode.png)
>![Scope settings](../pictures/results_plot_arduino_photodiode2.png)
>![Scope settings](../pictures/results_plot_photodiode_both.png)
>![Scope settings](../pictures/results_plot_arduino_both.png)